# Predicción del valor de mercado de autos usados: Construcción de un modelo utilizando datos históricos de automóviles para una función de la aplicación que puede determinar el valor de mercado del automóvil de un usuario por *Carlos Horta* (carlosgim@gmail.com)

---

## Introducción: Predicción del valor de mercado de autos usados

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Esta aplicación permitirá a los usuarios obtener rápidamente el valor de mercado de su automóvil. Para lograr esto, se requiere crear un modelo que pueda determinar el valor de mercado de los autos.

A Rusty Bargain le interesan los siguientes aspectos del modelo:

- **Calidad de la predicción:** El modelo debe ser capaz de realizar predicciones precisas y confiables del valor de mercado de los autos usados. Esto es crucial para brindar a los clientes información precisa y ayudarles a tomar decisiones informadas.

- **Velocidad de la predicción:** La aplicación busca proporcionar una respuesta rápida a los usuarios. Por lo tanto, es importante que el modelo sea eficiente en términos de tiempo de respuesta al realizar predicciones.

- **Tiempo requerido para el entrenamiento:** Además de la velocidad de predicción, Rusty Bargain también considera importante el tiempo necesario para entrenar el modelo. Un tiempo de entrenamiento más corto permitirá a la empresa iterar y actualizar el modelo con mayor agilidad.

Para cumplir con estos requisitos, se explorarán diferentes técnicas y algoritmos de Machine Learning que sean capaces de ofrecer una buena calidad de predicción, una velocidad de predicción rápida y un tiempo de entrenamiento eficiente. Se evaluarán y compararán varios modelos para seleccionar la opción que mejor se adapte a las necesidades de Rusty Bargain.

El objetivo final es desarrollar un modelo que pueda proporcionar a los usuarios estimaciones precisas y rápidas del valor de mercado de sus autos usados, mejorando así la experiencia del cliente y promoviendo la eficiencia en el proceso de venta de autos usados de Rusty Bargain.


## Preparación de datos

### Inicialización 

In [ ]:
# Es necesario cargar las diferentes librerías que se van a utilizar en el desarrollo del proyecto.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.spatial import distance
from numpy.random import RandomState
import plotly.graph_objs as go
import plotly.express as px
import time

In [ ]:
# Y definir la semilla aleatoria para que los resultados sean reproducibles.

rs = RandomState(seed=1984)

### Carga de los datos

In [ ]:
# Se cargan los datos de los ficheros csv en los dataframes correspondientes.

try:
    df = pd.read_csv('car_data.csv')
except:
    df = pd.read_csv('/datasets/car_data.csv')  

In [ ]:
# Se observan los datos del dataframe.

df.info()

En el DataFrame analizado, se observa que contiene un total de 354,369 registros y 16 columnas. Sin embargo, también se identificó la presencia de datos faltantes en algunas columnas específicas. Las columnas con datos faltantes son las siguientes: VehicleType, Gearbox, Model, FuelType, NotRepaired y NumberOfPictures.

Para mejorar la calidad y el estilo del DataFrame, se proponen las siguientes acciones inmediatas:

1. **Cambiar los nombres de las columnas a minúsculas**: Se sugiere convertir los nombres de las columnas a minúsculas para mantener la consistencia y facilitar la manipulación de los datos.

2. **Reescribir los títulos de acuerdo con las sugerencias de estilo**: Se recomienda revisar y ajustar los títulos de las columnas siguiendo las convenciones de estilo recomendadas, como utilizar letras minúsculas, separadores de palabras (por ejemplo, guiones bajos o camel case) y evitar espacios en blanco o caracteres especiales.

Estas acciones inmediatas ayudarán a mejorar la legibilidad, coherencia y uniformidad del DataFrame, facilitando su posterior análisis y manipulación de datos.

In [ ]:
df = df.rename(columns={'DateCrawled': 'date_crawled', 'Price': 'price', 'VehicleType': 'vehicle_type', 'RegistrationYear': 'registration_year', 'Gearbox': 'gearbox', 'Power': 'power', 'Model': 'model', 'Mileage': 'mileage', 'RegistrationMonth': 'registration_month', 'FuelType': 'fuel_type', 'Brand': 'brand', 'NotRepaired': 'not_repaired', 'DateCreated': 'date_created', 'NumberOfPictures': 'number_of_pictures', 'PostalCode': 'postal_code', 'LastSeen': 'last_seen'})

## Columnas numéricas del dataset

In [ ]:
# Es necesario observar los valores numéricos de las columnas del dataset.

df.describe()

### Columna 'number of pictures'
Como se observa en el análisis, la columna 'number of pictures' contiene únicamente valores de cero en todas las filas. Debido a que esta columna no aporta información relevante para el análisis y no presenta variabilidad, se puede eliminar sin afectar los resultados del proyecto en el siguiente apartado.

### Valores no válidos en columnas numéricas
Al analizar las columnas restantes con valores numéricos, se identificaron algunos valores que no tienen sentido y que podrían considerarse como datos erróneos. A continuación, se describen los problemas encontrados en cada columna:

- **Año de registro**: Se encontraron valores atípicos como 1000 y 9999, que no corresponden a años válidos para el registro de vehículos. Estos datos podrían ser considerados como errores o datos faltantes.

- **Potencia en CV**: Se identificaron valores extremos e incoherentes, como cero o cifras muy altas (por ejemplo, 20,000 CV), que no son posibles para la potencia de un vehículo. Estos valores también podrían ser considerados como errores o datos faltantes.

- **Mes de registro**: Se observaron filas con un valor de cero en el mes de registro, lo cual no es válido ya que el mes debe estar en el rango de 1 a 12. Estos datos podrían considerarse como errores o datos faltantes.

- **Millaje**: Se encontró que la mediana y los cuartiles tercero y cuarto están situados en el valor de 150,000 millas. Esto puede indicar que hay una alta concentración de datos en este valor, lo cual podría afectar la variabilidad de los datos y la capacidad del modelo para capturar diferencias significativas en el millaje de los vehículos.

En el siguiente apartado del proyecto se abordarán estas irregularidades y se tomarán las medidas adecuadas, como la limpieza de datos, para garantizar la calidad y la coherencia de los datos utilizados en el análisis.


### Columna 'registration_year'

In [ ]:
import pandas as pd
import plotly.graph_objs as go
import plotly.subplots as sp

# Filtramos los datos de la columna "registration_year"
df_filtered = df[(df['registration_year'] >= 1900) & (df['registration_year'] <= 2023)]

# Creamos el histograma y el boxplot
histogram = go.Histogram(x=df_filtered['registration_year'], nbinsx=50, name='Histograma')
boxplot = go.Box(x=df_filtered['registration_year'], name='Boxplot')

# Creamos un subplot con las dos figuras
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Histograma", "Boxplot"))
fig.add_trace(histogram, row=1, col=1)
fig.add_trace(boxplot, row=1, col=2)

# Ajustamos el layout y mostramos la figura
fig.update_layout(title='Análisis de la columna "registration_year"', showlegend=False)
fig.show()


En este apartado se analizó la columna 'registration_year', que consta de 354,369 filas. Sin embargo, se identificó que presenta valores extremos que no son posibles, con un valor mínimo de 1,000 y un valor máximo de 9,999. Estos valores están claramente fuera del rango válido para el año de registro de un vehículo.

Para abordar este problema, se realizó un gráfico que delimitó los años entre 1900 y 2023, con el fin de visualizar mejor la distribución de los datos. Sin embargo, incluso con esta delimitación, aún se observan datos extremos en el extremo izquierdo del gráfico.

**Decisión sobre la columna 'registration_year':** Para garantizar la integridad y la coherencia de los datos, se ha decidido eliminar los registros que se encuentren fuera del rango entre 1989 y 2019. Este rango se considera más realista y acorde con los años de registro de los vehículos en el contexto del proyecto.

Al realizar esta limpieza de datos, se asegurará que los valores en la columna 'registration_year' sean más coherentes y representativos de la realidad. Esta decisión contribuirá a mejorar la calidad de los datos y evitará la inclusión de valores extremos que puedan afectar el análisis y la precisión del modelo.

En el siguiente apartado del proyecto se aplicará esta decisión y se procederá a la limpieza de la columna 'registration_year' según el rango especificado.


### Columna 'power'

In [ ]:
# Crear el histograma
hist = go.Histogram(x=df['power'], name='Potencia')

# Crear el boxplot
box = go.Box(y=df['power'], name='Potencia')

# Crear el subplot
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Histograma', 'Boxplot'))
fig.add_trace(hist, row=1, col=1)
fig.add_trace(box, row=1, col=2)

# Establecer el layout
fig.update_layout(width=900, height=400, title='Análisis de la columna "power"')

# Mostrar la figura
fig.show()

La potencia en CV (caballos de vapor) es una medida utilizada para determinar la potencia del motor de un vehículo. En general, los valores de potencia en CV se encuentran dentro de un rango específico según el tipo y tamaño del vehículo. Por ejemplo, los carros urbanos suelen tener motores de al menos 60 CV, mientras que los carros grandes de alto rendimiento pueden llegar a tener potencias superiores a los 1,000 CV.

Al analizar la columna 'power' en el gráfico anterior, se observa una distribución de datos que alcanza un máximo de 20,000 CV. Sin embargo, esta cifra es extremadamente alta y poco realista en el contexto de los vehículos utilizados en el proyecto.

**Decisión sobre la columna 'power':** Con el objetivo de mantener la coherencia y evitar valores extremos que puedan afectar el análisis y la precisión del modelo, se ha decidido eliminar los datos de aquellas filas cuyo valor en la columna 'power' supere los 2,000 caballos de vapor. Esta decisión se basa en consideraciones realistas y en el rango de potencia habitual de los vehículos utilizados en el mercado.

Al aplicar esta limpieza de datos, se garantizará que los valores en la columna 'power' sean más consistentes y representativos de los motores de los vehículos. Esto contribuirá a mejorar la calidad de los datos y evitará la inclusión de valores extremos que puedan distorsionar el análisis y los resultados del modelo.

En el siguiente apartado del proyecto se llevará a cabo esta acción y se procederá a la limpieza de la columna 'power' eliminando las filas con potencias superiores a los 2,000 caballos de vapor.


### Columna 'mileage'

In [ ]:
# Crear el histograma
hist = go.Histogram(x=df['mileage'], name='Kilometraje')

# Crear el boxplot
box = go.Box(y=df['mileage'], name='Kilometraje')

# Crear el subplot
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Histograma', 'Boxplot'))
fig.add_trace(hist, row=1, col=1)
fig.add_trace(box, row=1, col=2)

# Establecer el layout
fig.update_layout(width=900, height=400, title='Análisis de la columna "mileage"')

# Mostrar la figura
fig.show()

Al analizar la columna 'mileage', se observa una tendencia particular en los datos. Un alto porcentaje de los vehículos tiene un kilometraje registrado de 150,000 kilómetros, lo cual puede resultar sospechoso. Es posible que esto se deba a que, al obtener los datos de esta columna, se registraron valores aproximados en números redondos en lugar de valores exactos.

**Decisión sobre la columna 'mileage':** Después de considerar esta observación, se ha decidido mantener los datos tal como se encuentran en el dataset de este proyecto. Aunque existe una aparente estandarización en los valores de kilometraje, se asumirá que estos datos reflejan la información proporcionada y registrada en el sitio Rusty Bargain.

No se realizará ninguna modificación ni limpieza adicional en la columna 'mileage' para evitar la introducción de sesgos o pérdida de información en el análisis y en el modelo de predicción del valor de mercado de los vehículos.

En el siguiente apartado del proyecto se trabajará con la columna 'mileage' sin realizar modificaciones en sus datos.


### Columna 'registration_month'

In [ ]:
# Crea una columna booleana que indica si el valor de registration_month es cero
df["is_zero"] = (df["registration_month"] == 0)

# Crea el gráfico, utilizando la columna "is_zero" para marcar las filas con valor cero
fig = px.histogram(df, x="registration_month", nbins=24, color="is_zero")

# Muestra el gráfico
fig.show()

Al analizar la columna 'registration_month', se observa la presencia de 37,532 filas con el valor cero en esta columna. El valor cero no es válido para representar un mes de registro, ya que los meses van del 1 al 12.

**Decisión sobre la columna 'registration_month':** Para mantener la coherencia y consistencia de los datos, se ha decidido eliminar las filas que contengan el valor cero en la columna 'registration_month'. Estas filas no aportan información válida y podrían afectar el análisis y los resultados del modelo.

Al eliminar estas filas, se asegurará que los valores en la columna 'registration_month' sean coherentes y se correspondan con meses válidos, lo que contribuirá a mejorar la calidad de los datos utilizados en el proyecto.

En el siguiente apartado del proyecto se aplicará esta decisión y se procederá a la eliminación de las filas que contengan el valor cero en la columna 'registration_month'.

## En resumen sobres las columnas numéricas:

### Resumen sobre las columnas numéricas

Después de analizar las columnas numéricas, se tomaron las siguientes decisiones para cada una de ellas:

- **Columna 'number_of_pictures':** Se eliminó la columna en su totalidad, ya que todas las filas contenían únicamente ceros, lo que la hacía irrelevante para el análisis.

- **Columna 'registration_year':** Se eliminaron los datos anteriores a 1989 y posteriores a 2019, ya que se consideraban valores atípicos o incorrectos para el año de registro de un vehículo.

- **Columna 'power':** Se eliminaron las filas que contenían valores de potencia superiores a 2,000 caballos de vapor, ya que se consideraban extremadamente altos y poco realistas.

- **Columna 'mileage':** Los datos se dejaron tal como estaban en el dataset, ya que no se identificaron problemas significativos o datos incorrectos en esta columna.

- **Columna 'postal_code':** La columna se eliminó, ya que no era relevante para el objetivo del proyecto.

- **Columna 'registration_month':** Se eliminaron las filas que contenían el valor cero en esta columna, ya que no correspondían a un mes válido de registro.

- **Columnas 'date_crawled', 'date_created' y 'last_seen':** Estas columnas se eliminaron, ya que contenían información no relevante para el objetivo del proyecto.

Estas acciones ya se han realizado y se ha llevado a cabo la modificación correspondiente en el dataset.

### Trabajo de eliminación de filas y/o columnas numéricas

In [ ]:
# Antes de iniciar la eliminación de valores y columnas numéricas, es necesario realizar una copia del dataframe original.

df_copia = df.copy()

In [ ]:
# Se procede a realizar la eliminación de los valores y columnas numéricas que se describió en el apartado anterior.

# Eliminar la columna 'number_of_pictures', 'postal_code', 'date_crawled', 'date_created', 'last_seen'

df = df.drop(['number_of_pictures', 'postal_code', 'date_crawled', 'date_created', 'last_seen'], axis=1)

# De la columna 'registration_year' se conservan los valores que se encuentren entre los años 1989 y 2019.
df = df.loc[(df['registration_year'] >= 1988) & (df['registration_year'] <= 2020)]

# De la columna 'power' se eliminarán los valores que se encuentren a partir de 2000 de caballos de vapor.
df = df.loc[df['power'] < 2000]

# De la columna 'registration_month' se eliminarán los valores que tengan valor cero.   
df = df.loc[df['registration_month'] != 0]

# Hasta este momento los datos del dataframe se encuentran en el siguiente estado.
df.info()

Como se observa, los datos de iniciales del dataset era de 354,369 por columna y ahora es de 311,147. En el siguiente apartado se revisarán cada una de las columnas categóricas. 

## Columnas categóricas del dataset

En esta sección se revisarán las columnas categóricas del dataset del proyecto, que incluyen información sobre el tipo de vehículo, el tipo de caja de cambios, el modelo del automóvil, el tipo de combustible, la marca del vehículo y si ha sido reparado o no.

Se realizará un análisis de cada una de estas columnas para identificar patrones, valores únicos y posibles acciones a tomar en cada caso.

### Columna "vehicle_type"

In [ ]:
print(df['vehicle_type'].value_counts())
print('---------------------------------')
print(df['vehicle_type'].isna().sum())

Después de analizar la columna 'vehicle_type', se encontraron 22,267 valores faltantes. Una opción adecuada para incluir estos datos ausentes es utilizar la categoría 'other' (otros), ya que proporciona una clasificación general para los vehículos que no se pueden identificar correctamente.

**Decisión sobre la columna 'vehicle_type':** Se ha decidido asignar los datos faltantes a la categoría 'other', ya que es la opción más adecuada para los valores ausentes en esta columna. Esto permitirá mantener la integridad de los datos y asegurará que todas las filas tengan una clasificación válida en términos de tipo de vehículo.



### Columna 'gearbox'

In [ ]:
print(df['gearbox'].value_counts())
print('---------------------------------')
print(df['gearbox'].isna().sum())

Al analizar la columna 'gearbox', se observa que existen dos categorías: 'manual' y 'automático'. Sin embargo, se encontraron 9,488 valores faltantes en esta columna.

**Decisión sobre la columna 'gearbox':** Dado que la categoría 'manual' es la más frecuente en el dataset (240,794 filas) en comparación con la categoría 'automático' (60,865 filas), se ha decidido imputar los valores faltantes en la categoría 'manual'. Esta decisión se basa en la frecuencia y en el hecho de que 'manual' es la opción más común para la mayoría de los automóviles listados.

### Columna 'model'

In [ ]:
print(df['model'].value_counts())
print('---------------------------------')
print(df['model'].isna().sum())
print('---------------------------------')
print(df['model'].nunique())

Después de analizar la columna 'model', se encontró que hay 248 valores únicos en esta columna. Esto podría aumentar la complejidad del modelo y potencialmente generar problemas de sobreajuste. Además, si se utiliza una técnica de codificación como la codificación one-hot, se generarían 248 columnas adicionales, lo que a su vez aumentaría el número de parámetros del modelo y podría afectar su capacidad de generalización.

**Decisión sobre la columna 'model':** Con el objetivo de evitar problemas de sobreajuste y mejorar la capacidad del modelo para generalizar a nuevos datos, se ha decidido eliminar la columna 'model' del conjunto de datos. Esta acción reducirá la complejidad del modelo y permitirá un enfoque más sencillo en la predicción del valor de mercado de los automóviles.

### Columna 'fuel_type'

In [ ]:
print(df['fuel_type'].value_counts())
print('---------------------------------')
print(df['fuel_type'].isna().sum())
print('---------------------------------')
print(df['fuel_type'].nunique())

Después de analizar la columna 'fuel_type', se encontraron datos faltantes en esta columna. Además, se observó que hay 7 valores posibles en total para indicar el tipo de combustible utilizado por el automóvil.

**Decisión sobre la columna 'fuel_type':** Para conservar la información disponible en los registros y evitar la eliminación de datos importantes, se ha decidido imputar los valores faltantes en la categoría 'other'. Esta opción permite agrupar los datos ausentes en una categoría general que representa otros tipos de combustible no especificados en la lista original.

### Columna 'brand'

In [ ]:
print(df['brand'].value_counts())
print('---------------------------------')
print(df['brand'].isna().sum())
print('---------------------------------')
print(df['brand'].nunique())

Después de analizar la columna 'brand', se encontraron 40 marcas únicas dentro de las posibles respuestas, y no existen datos ausentes en ella. Es importante tener en cuenta que, al tener un gran número de valores únicos en una columna, puede aumentar la posibilidad de sobreajuste del modelo.

Después de evaluar el impacto que tiene la columna 'brand' en el modelo y considerar opciones para reducir su dimensionalidad, se ha decidido eliminar la columna del dataset para evitar el sobreajuste del modelo. Aunque existe la codificación de hash (Hash Encoding) y la codificación de embeddings (Embedding Encoding) para convertir variables categóricas en numéricas, se considera que eliminar la columna 'brand' es la mejor opción en este caso.

Al eliminar la columna 'brand', se reduce la cantidad de características que se utilizan en el modelo, lo que puede mejorar la capacidad de generalización del modelo a datos nuevos e independientes. También se simplifica la tarea de encoding y se reduce el riesgo de sobreajuste.

### Columna 'not_repaired'

In [ ]:
print(df['not_repaired'].value_counts())
print('---------------------------------')
print(df['not_repaired'].isna().sum())
print('---------------------------------')
print(df['not_repaired'].nunique())

En relación con la columna 'not_repaired', se encontraron dos posibles respuestas: "sí" y "no". Sin embargo, se identificaron 47,733 valores ausentes en esta columna.

**Decisión sobre la columna 'not_repaired':** Para imputar los datos faltantes, se ha decidido asignar el valor "no" a los registros faltantes. Esta decisión se basa en la hipótesis de que los registros sin información disponible probablemente no hayan sido reparados, ya que no hay indicación de reparación en los datos.

## En resumen sobres las columnas categóricas:

Después del análisis de las columnas categóricas, se han tomado las siguientes decisiones para cada una de ellas:

**Decisión sobre la columna *'vehicle_type*:** Se ha decidido colocar los datos ausentes dentro de la categoría 'other', ya que es la opción más adecuada para este tipo de datos faltantes.

**Decisión sobre la columna *'gearbox*:** Los datos ausentes se imputarán en la categoría de 'manual', ya que es el valor más frecuente en el conjunto de datos (240,794 filas versus 60,865 de automáticos).

**Decisión sobre la columna *'model*:** Se eliminará esta columna del conjunto de datos para evitar problemas de sobreajuste y mejorar la capacidad del modelo para generalizar a nuevos datos.

**Decisión sobre la columna *'fuel_type*:** Los datos ausentes se imputarán dentro de la categoría 'other'.

**Decisión sobre la columna *'brand*:** Se ha decidido eliminar la columna para reducir la cantidad de características en el modelo.

**Decisión sobre la columna *'not_repaired*:** Se ha decidido asignar el valor 'no' a los registros faltantes, ya que es la categoría más probable para los vehículos sin información de reparaciones.

Estas decisiones se aplicarán a continuación.

### Trabajo de eliminación de filas y/o columnas categóricas

In [ ]:
# Se procede a realizar la eliminación de los datos correspondiente a filas y/o columnas categóricas.

# En 'vehicle_type' los valores ausentes se imputarán con el valor 'other'.
df['vehicle_type'].fillna(value='other', axis=0, inplace=True)

# En 'gearbox' los valores ausentes se imputarán con el valor 'manual'.
df['gearbox'].fillna(value='manual', axis=0, inplace=True)

# En 'fuel_type' los valores ausentes se imputarán con el valor 'other'.
df['fuel_type'].fillna(value='other', axis=0, inplace=True)

# En 'not_repaired' los valores ausentes se imputarán con el valor 'no'.
df['not_repaired'].fillna(value='no', axis=0, inplace=True)

In [ ]:
# Por último, la columna 'model' y 'brand' se eliminarán del dataframe.

df = df.drop(['model'], axis=1)
df = df.drop(['brand'], axis=1)

# Y también la columna 'is_zero' que se creó para el análisis de la columna 'registration_month'.

df = df.drop(['is_zero'], axis=1)

In [ ]:
# Así se encuentra el dataset después de la limpieza de datos.
df.info()

In [ ]:
# Se realizará una copia del dataset para cualquier eventualidad.

df_clean = df.copy()

## Datos con OrdinalEncoder

In [ ]:
enc = OrdinalEncoder()
df_enc = df_clean.copy()
df_enc[['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired']] = enc.fit_transform(df_clean[['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired']])

# Se crean las variables dummies para las columnas 'vehicle_type', 'gearbox', 'fuel_type', 'not_repaired'.
df_ohe = pd.get_dummies(df_clean, columns=['vehicle_type', 'gearbox', 'fuel_type', 'not_repaired'], drop_first=True)

df_ohe.shape

### Creación de features y target

In [ ]:
# Se separan los datos para la validación del modelo.
rest_enc, valid = train_test_split(df_enc, test_size=0.2, random_state=rs)

# También para el entrenamiento y prueba del modelo.
train, test = train_test_split(rest_enc, test_size=0.25, random_state=rs)

# Se crean las variables objetivo y predictores para el entrenamiento, prueba y validación del modelo.
features_train_enc = train.drop(['price'], axis=1)
target_train_enc = train['price']

features_test_enc = test.drop(['price'], axis=1)
target_test_enc = test['price']

features_valid_enc = valid.drop(['price'], axis=1)
target_valid_enc = valid['price'] 

# Y para el entrenamiento de la validación.
features_rest_enc = rest_enc.drop(['price'], axis=1)
target_rest_enc = rest_enc['price']

# Se comprueban los tamaño de los conjuntos de datos.
print(features_train_enc.shape)
print(target_train_enc.shape)
print(features_test_enc.shape)
print(target_test_enc.shape)
print(features_valid_enc.shape)
print(target_valid_enc.shape)

## Datos con OneHotEncoder


In [ ]:
# Se separan los datos para la validación del modelo.
rest_ohe, valid = train_test_split(df_ohe, test_size=0.2, random_state=rs)

# También para el entrenamiento y prueba del modelo.
train, test = train_test_split(rest_ohe, test_size=0.25, random_state=rs)

# Se crean las variables objetivo y predictores para el entrenamiento, prueba y validación del modelo.
features_train_ohe = train.drop(['price'], axis=1)
target_train_ohe = train['price']

features_test_ohe = test.drop(['price'], axis=1)
target_test_ohe = test['price']

features_valid_ohe = valid.drop(['price'], axis=1)
target_valid_ohe = valid['price']

# Y para el entrenamiento de la validación.
features_rest_ohe = rest_ohe.drop(['price'], axis=1)
target_rest_ohe = rest_ohe['price']

# Se comprueban los tamaño de los conjuntos de datos.
print(features_train_ohe.shape)
print(target_train_ohe.shape)
print(features_test_ohe.shape)
print(target_test_ohe.shape)
print(features_valid_ohe.shape)
print(target_valid_ohe.shape)


## Datos sin codificar

In [ ]:
# Se separan los datos para la validación del modelo con datos sin codificar.
rest, valid = train_test_split(df, test_size=0.2, random_state=rs)

# También para el entrenamiento y prueba del modelo.
train, test = train_test_split(rest, test_size=0.25, random_state=rs)

# Se crean las variables objetivo y predictores para el entrenamiento, prueba y validación del modelo.
features_train = train.drop(['price'], axis=1)
target_train = train['price']

features_test = test.drop(['price'], axis=1)
target_test = test['price']

features_valid = valid.drop(['price'], axis=1)
target_valid = valid['price']

# Y para el entrenamiento de la validación.
features_rest = rest.drop(['price'], axis=1)
target_rest = rest['price']

# Se comprueban los tamaño de los conjuntos de datos.
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)


## Entrenamiento del modelo 

### Prueba de cordura con regresión lineal

In [ ]:
reg = LinearRegression()

# Se entrena el modelo y se mide el tiempo
start_time = time.time()
reg.fit(features_train_ohe, target_train_ohe)
train_time = time.time() - start_time

# Se hacen predicciones y se mide el tiempo
start_time = time.time()
reg_pred_ohe = reg.predict(features_valid_ohe)
pred_time = time.time() - start_time

# Se calcula el RECM
rmse = mean_squared_error(target_valid_ohe, reg_pred_ohe, squared=False)

# Se imprimien los resultados
print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

### Bosque aleatorio con ajuste de parámetros

In [ ]:
# Se configuran parámetros
best_score = float('inf')
n = 1

# Se entrena y evalúa el modelo para diferentes parámetros
for roots in range(3, 8):
    for leafs in range(3, 8):
        print(f'Ronda {n}')
        
        # Se crea el modelo y se miden tiempos de entrenamiento
        forest = RandomForestRegressor(random_state=rs, max_depth=roots, min_samples_leaf=leafs)
        start_time = time.time()
        forest.fit(features_train_enc, target_train_enc)
        train_time = time.time() - start_time
        
        # Se hacen predicciones y se mide el tiempo de predicción
        start_time = time.time()
        f_pred = forest.predict(features_valid_enc)
        pred_time = time.time() - start_time
        
        # Se calcula el RECM
        rmse = mean_squared_error(target_valid_enc, f_pred, squared=False)
        print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
        print(f'Tiempo de predicción: {pred_time:.2f} segundos')
        print(f'RECM: {rmse:.2f}')
        print()
        
        # Y se guarda el modelo con el mejor RECM
        if rmse < best_score:
            best_score = rmse
            best_root = roots
            best_leaf = leafs
            best_pred = f_pred
            best_n = n
            
        n += 1

# Entrenar modelo con los mejores parámetros
best_forest = RandomForestRegressor(random_state=rs, max_depth=best_root, min_samples_leaf=best_leaf)

# Imprimir resultados
print(f'Mejor ronda: {best_n}')
print(f'Mejor RECM: {best_score:.2f}')


Como se observa, la mejor ronda fue la 24 con un RECM de 2193.81 que mejor el RECM de la prueba de cordura que fue de 2854.45.

### CatBoost

In [ ]:
# Crear modelo y medir tiempo de entrenamiento
cat = CatBoostRegressor(iterations=200, cat_features=['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox'])
start_time = time.time()
cat.fit(features_train, target_train, eval_set=(features_valid, target_valid))
train_time = time.time() - start_time

# Hacer predicciones y medir tiempo de predicción
start_time = time.time()
cat_pred = cat.predict(features_valid)
pred_time = time.time() - start_time

# Calcular RECM
rmse = mean_squared_error(target_valid, cat_pred, squared=False)

# Imprimir resultados
print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

Con el CatBoost se mejoró la RECM y ahora se ubica en 1936.89 (versus 2193.81 del bosque aleatorio y de 2854.45 de la regresión lineal).

### LightGBM con ajuste de hiperparámetros

In [ ]:
# Con esta función se convierten columnas a tipo 'category'
def convert_columns_to_category(df, columns):
    for column in columns:
        df[column] = df[column].astype('category')
    return df

# Luego se convierten columnas categóricas para cada conjunto de datos
categorical_columns = ['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox']
features_train = convert_columns_to_category(features_train, categorical_columns)
features_valid = convert_columns_to_category(features_valid, categorical_columns)
features_test = convert_columns_to_category(features_test, categorical_columns)
features_rest = convert_columns_to_category(features_rest, categorical_columns)

# Se crea el modelo y se miden el tiempo de entrenamiento
lgbm = lgb.LGBMRegressor(random_state=rs, learning_rate=0.05, n_estimators=500,
                         max_depth=5, num_leaves=31, min_child_samples=20,
                         subsample=0.8, colsample_bytree=0.8)

start_time = time.time()
lgbm.fit(features_train, target_train, eval_metric='RMSE',
         categorical_feature=categorical_columns, eval_set=(features_valid, target_valid))
train_time = time.time() - start_time

# Se hacen las predicciones y se mide el tiempo de predicción
start_time = time.time()
lgbm_pred = lgbm.predict(features_valid)
pred_time = time.time() - start_time

# Se calcula el RECM
rmse = mean_squared_error(target_valid, lgbm_pred, squared=False)

# Se imprimen los resultados
print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')
print(f'Tiempo de predicción: {pred_time:.2f} segundos')
print(f'RECM: {rmse:.2f}')

## Análisis del modelo

Para tomar una decisión sobre qué modelo es el mejor, debemos considerar tanto el rendimiento como el tiempo de computación.

Veamos los números:

**Bosque aleatorio:** El tiempo de entrenamiento es bastante largo (22.82 segundos) y el RECM es el segundo más alto (2193.81).

**Regresión lineal:** El tiempo de entrenamiento y predicción es el más bajo, pero también es el que tiene el RECM más alto (2854.45), lo que significa que es el menos preciso.

**CatBoost:** Este tiene el RECM más bajo de todos los modelos (1936.89), lo que indica que es el más preciso. Aunque su tiempo de entrenamiento es alto (22.39 segundos), es ligeramente menor que el del Bosque Aleatorio. Además, su tiempo de predicción es significativamente más rápido que el de los otros modelos (0.10 segundos).

**LightGBM:** Aunque tiene un RECM ligeramente más alto que el CatBoost (1944.44), su tiempo de entrenamiento es significativamente más corto (2.81 segundos), lo que puede ser una ventaja dependiendo de las circunstancias.

Si consideramos solo el rendimiento, el modelo CatBoost sería el mejor, ya que tiene el RECM más bajo, lo que indica que es el más preciso.

Sin embargo, si también consideramos el tiempo de entrenamiento y predicción, LightGBM podría ser una opción mejor. Aunque su RECM es ligeramente más alto que el de CatBoost, su tiempo de entrenamiento es mucho más corto, lo que podría ser una ventaja si necesitamos entrenar el modelo con frecuencia o si trabajamos con un conjunto de datos muy grande.

Para este proyecto se eligirá como mejor modelo al **LightGBM** porque tiene mejor equilibrio entre precisión y eficiencia computacional.

In [ ]:
# Se crea el modelo
lgbm = lgb.LGBMRegressor(random_state=rs)

# Se mide tiempo de entrenamiento
start_time = time.time()
lgbm.fit(features_rest, target_rest, eval_metric = 'RMSE', 
         categorical_feature=['vehicle_type', 'fuel_type', 'not_repaired', 'gearbox'], 
         eval_set=(features_test, target_test))
train_time = time.time() - start_time

print(f'Tiempo de entrenamiento: {train_time:.2f} segundos')

# Se midel el tiempo de predicción
start_time = time.time()
lgbm_pred = lgbm.predict(features_test)
pred_time = time.time() - start_time

print(f'Tiempo de predicción: {pred_time:.2f} segundos')

# Se calcula e imprime el RECM
rmse = mean_squared_error(target_test, lgbm_pred, squared=False)
print(f'RECM: {rmse:.2f}')


| Iteración | RMSE    |
|----------:|--------:|
| 1         | 4241.63 |
| 2         | 3957.93 |
| 3         | 3710.92 |
| 4         | 3489.42 |
| 5         | 3298.87 |
| 6         | 3135.63 |
| 7         | 2990.74 |
| 8         | 2865.51 |
| 9         | 2757.10 |
| 10        | 2664.85 |
| 11        | 2581.47 |
| 12        | 2508.86 |
| 13        | 2447.89 |
| 14        | 2394.33 |
| 15        | 2348.26 |
| 16        | 2308.58 |
| 17        | 2273.39 |
| 18        | 2243.22 |
| 19        | 2217.47 |
| 20        | 2194.01 |
| 21        | 2173.84 |
| 22        | 2156.77 |
| 23        | 2139.97 |
| 24        | 2125.84 |
| 25        | 2111.86 |
| 100       | 1924.40 |


Como se puede observar, el tiempo de entrenamiento con los datos de validación pasó de 2.81 segundos a 1.05 segundos, mientras que el tiempo de predicción disminuyó de 0.32 segundos a 0.07 segundos. Además, el RECM se redujo de 1944.44 a 1924.40. Estos resultados indican que el modelo LightGBM logró mejorar tanto los tiempos de ejecución como la precisión de las predicciones.

En conclusión, el modelo LightGBM resultó ser una buena opción para mejorar los tiempos de ejecución y predicción en comparación con los otros modelos evaluados.
